In [1]:
import numpy as np
import pandas as pd
import xarray as xr
from joblib import dump, load
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import glob

import matplotlib as mpl

import matplotlib.pyplot as plt

import math as mt

import cartopy.crs as ccrs

import cartopy.feature as cfeature

from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

#AO Calculation

In [2]:
# Calculate AO - input std normal latitude wighted 1000mb z
ZData = xr.open_dataset('../../../Data/CMIP6/MPI-ESM1-2-HR/processed/weighted1000Std.nc')
#SLPData = xr.open_dataset('../eraDown/ERA5_2degree_Down/DailyMean/weighted1000.nc')

ZData.zg.values
ZData.zg.values=xr.where(np.isnan(ZData.zg.values),  0.000000000001,ZData.zg.values)
ZData.zg.values

array([[[168.2589   , 170.89351  , 175.64795  , ..., 166.16936  ,
         165.73026  , 166.56305  ],
        [172.81638  , 173.84285  , 177.01285  , ..., 172.55975  ,
         171.51819  , 171.95595  ],
        [179.25833  , 177.81384  , 177.94926  , ..., 179.95049  ,
         179.21318  , 179.69469  ],
        ...,
        [ 35.2512   ,  35.237953 ,  35.244576 , ...,  35.241264 ,
          35.254513 ,  35.257824 ],
        [ 26.852762 ,  26.858055 ,  26.850117 , ...,  26.879221 ,
          26.865992 ,  26.873928 ],
        [ 16.75734  ,  16.760834 ,  16.7521   , ...,  16.760834 ,
          16.75734  ,  16.7521   ]],

       [[159.85748  , 163.02206  , 168.18532  , ..., 152.6804   ,
         154.86078  , 157.4197   ],
        [162.97641  , 165.5426   , 169.92021  , ..., 155.67033  ,
         157.57233  , 160.33475  ],
        [166.22995  , 167.79482  , 170.11201  , ..., 158.19499  ,
         160.33162  , 163.56668  ],
        ...,
        [ 27.418009 ,  27.52069  ,  27.646559 , ...,  

In [3]:


t1=ZData.zg.stack(z=("lat", "lon"))
# fit scaler on training data
zg_PC =PCA().fit_transform(t1)


In [4]:
print(zg_PC[:,0].shape)
AO_ds = xr.Dataset({'AO': (('time'), zg_PC[:,0])}, coords={'time': ZData.time})


norm = StandardScaler().fit(zg_PC[:,0].reshape(-1, 1))
    # transform training data
aoX = norm.transform(zg_PC[:,0].reshape(-1, 1))
aoX.shape

AO_ds = xr.Dataset({'AO': (('time'), aoX[:,0]*-1.0)}, coords={'time': ZData.time})


(4810,)


In [5]:
AO_ds.to_netcdf('MPI-ESM1-2-HR-AOindex-NDJF-Daily-1980-2014.nc')

In [6]:
AO_ds.AO.where(AO_ds.AO==AO_ds.AO.max(), drop=True).squeeze()

<xarray.DataArray 'AO' ()>
array(3.618459, dtype=float32)
Coordinates:
    time     datetime64[ns] 2000-02-02T12:00:00

In [7]:
#AO_ds.AO.where(AO_ds.AO>2.0, drop=True).squeeze()

In [8]:
AO_ds.AO.where(AO_ds.AO==AO_ds.AO.min(), drop=True).squeeze()

<xarray.DataArray 'AO' ()>
array(-3.6652944, dtype=float32)
Coordinates:
    time     datetime64[ns] 1988-12-17T12:00:00

In [9]:
#AO_ds.AO.where(AO_ds.AO<-2, drop=True).squeeze()

#EU Calculation

In [10]:
zLevData = xr.open_dataset('../../../Data/CMIP6/MPI-ESM1-2-HR/processed/NDJF_New_zg_day_MPI-ESM1-2-HR_historical_r1i1p1f1_gn_18500101-20091231.nc')

In [11]:
zLevData.plev

<xarray.DataArray 'plev' (plev: 8)>
array([100000.,  85000.,  70000.,  50000.,  25000.,  10000.,   5000.,   1000.])
Coordinates:
  * plev     (plev) float64 1e+05 8.5e+04 7e+04 5e+04 2.5e+04 1e+04 5e+03 1e+03
Attributes:
    standard_name:  air_pressure
    long_name:      pressure
    units:          Pa
    positive:       down
    axis:           Z

In [12]:
z500=zLevData.zg.sel(plev=50000.0,method='nearest')
z500

<xarray.DataArray 'zg' (time: 4810, lat: 192, lon: 384)>
[354631680 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 1975-01-01T12:00:00 ... 2014-12-31T12:00:00
  * lon      (lon) float64 0.0 0.9375 1.875 2.812 ... 356.2 357.2 358.1 359.1
  * lat      (lat) float64 -89.28 -88.36 -87.42 -86.49 ... 87.42 88.36 89.28
    plev     float64 5e+04
Attributes:
    standard_name:  geopotential_height
    long_name:      Geopotential Height
    units:          m
    comment:        Geopotential is the sum of the specific gravitational pot...
    cell_methods:   time: mean
    cell_measures:  area: areacella
    history:        2019-08-25T06:42:13Z altered by CMOR: Reordered dimension...

In [13]:
z145E40N=z500.sel(lat=40,lon=145,method='nearest')

In [14]:
#z145E40N

In [15]:
z20E55N=z500.sel(lat=55,lon=20,method='nearest')

In [16]:
#z20E55N

In [17]:
z75E55N=z500.sel(lat=55,lon=75,method='nearest')

In [18]:
#z75E55N

In [19]:
#Calculate Anomaly
z145E40N_Anom = z145E40N.groupby("time.dayofyear") - z145E40N.groupby("time.dayofyear").mean("time")
z75E55N_Anom  = z75E55N.groupby("time.dayofyear")  - z75E55N.groupby("time.dayofyear").mean("time")
z20E55N_Anom  = z20E55N.groupby("time.dayofyear")  - z20E55N.groupby("time.dayofyear").mean("time")

In [20]:
# calculate std normal anomalies
z145E40N_AnomStd = z145E40N_Anom.groupby("time.dayofyear")  / z145E40N.groupby("time.dayofyear").std("time")
z75E55N_AnomStd  = z75E55N_Anom.groupby("time.dayofyear")   / z75E55N.groupby("time.dayofyear").std("time")
z20E55N_AnomStd  = z20E55N_Anom.groupby("time.dayofyear")   / z20E55N.groupby("time.dayofyear").std("time")

In [21]:
EUVal = -(1.0/4.0)*z20E55N_AnomStd + (1.0/2.0)*z75E55N_AnomStd - (1.0/4.0)*z145E40N_AnomStd

In [22]:
EUVal

<xarray.DataArray 'zg' (time: 4810)>
array([ 1.16463766,  0.98076392,  0.0730339 , ..., -1.15338277,
       -0.97989992, -0.68891561])
Coordinates:
  * time       (time) datetime64[ns] 1975-01-01T12:00:00 ... 2014-12-31T12:00:00
    plev       float64 5e+04
    dayofyear  (time) int64 1 2 3 4 5 6 7 8 ... 358 359 360 361 362 363 364 365
    lon        float64 145.3

In [23]:
EU_ds = xr.Dataset({'EU': (('time'), EUVal)}, coords={'time': zLevData.time})

In [24]:
EU_ds.to_netcdf('MPI-ESM1-2-HR-EUindex-NDJF-Daily-1980-2014.nc')